<a href="https://colab.research.google.com/github/gommungommun/VCD/blob/master/DataLoader_ICDAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
!ls "/content/drive/MyDrive"

'Colab Notebooks'			      '과학 교양서 리스트_rev2.xlsx'  '제목 없는 문서.gdoc'
'갓준표 독서 소모임 과학 교양서 목록.gsheet'   보고서.gdoc


In [17]:
%cd /content/drive
!unzip -qq "/content/drive/MyDrive/kraggle_Data.zip"

Streaming output truncated to the last 5000 lines.
                 Operation not supported
                 unable to process data2/training_data/X/46309.png.
checkdir error:  cannot create data2
                 Operation not supported
                 unable to process data2/training_data/X/46345.png.
checkdir error:  cannot create data2
                 Operation not supported
                 unable to process data2/training_data/X/46381.png.
checkdir error:  cannot create data2
                 Operation not supported
                 unable to process data2/training_data/X/46417.png.
checkdir error:  cannot create data2
                 Operation not supported
                 unable to process data2/training_data/X/46453.png.
checkdir error:  cannot create data2
                 Operation not supported
                 unable to process data2/training_data/X/46489.png.
checkdir error:  cannot create data2
                 Operation not supported
                 unable to proce

In [22]:
!pip install kaggle

In [39]:
!echo '{"username":"seumgo","key":"3948cc52334d7483ef47dcedb05d31d6"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [40]:
!kaggle competitions download -c humpback-whale-identification

403 - Forbidden - You must accept this competition's rules before you'll be able to download files.


In [36]:
!mkdir -p ~/.kaggle

In [37]:
from google.colab import files
files.upload()

!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


OSError: [Errno 95] Operation not supported: 'kaggle.json'

In [33]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


OSError: [Errno 95] Operation not supported: 'kaggle.json'

In [30]:
# Make a directory named kaggle and copy the kaggle. json file there
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
# change the permission of the file
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [31]:
!kaggle competitions download -c humpback-whale-identification

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


##1. Data를 불러오는데 필요한 라이브러리 import


In [16]:
import os
import cv2
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [19]:
class ICDAR2015Dataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (str): 데이터셋의 루트 디렉토리 경로
            transform (callable, optional): 이미지에 적용할 변환 함수
        """
        self.root_dir = root_dir
        self.transform = transform
        self.images, self.gt_boxes, self.gt_texts = self.load_icdar2015(root_dir)

    def load_icdar2015(self, root_dir):
        images = []
        gt_boxes = []
        gt_texts = []

        img_dir = os.path.join(root_dir, 'ch4_training_images')
        gt_dir = os.path.join(root_dir, 'ch4_training_localization_transcription_gt')

        image_files = sorted(os.listdir(img_dir))

        for img_file in image_files:
            img_path = os.path.join(img_dir, img_file)
            image = cv2.imread(img_path)
            images.append(image)

            gtfile = 'gt' + img_file.replace('.jpg', '.txt')
            gt_path = os.path.join(gt_dir, gt_file)

            boxes = []
            texts = []

            with open(gt_path, 'r', encoding='utf-8-sig') as f:
                for line in f:
                    parts = line.strip().split(',')
                    coords = list(map(float, parts[:8]))
                    box = np.array(coords).reshape(-1, 2)
                    boxes.append(box)
                    text = ','.join(parts[8:])
                    texts.append(text)

            gt_boxes.append(boxes)
            gt_texts.append(texts)

        return images, gt_boxes, gt_texts

    def len(self):
        return len(self.images)

    def getitem(self, idx):
        image = self.images[idx]
        boxes = self.gt_boxes[idx]
        texts = self.gt_texts[idx]

        if self.transform:
            image = self.transform(image)

        sample = {'image': image, 'boxes': boxes, 'texts': texts}

        return sample

In [21]:
root_dir = '/content/drive/kraggle_Data'

# 변환 함수 정의 (예: 이미지를 텐서로 변환)
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((640, 640)),
    transforms.ToTensor()
])

# 데이터셋 인스턴스 생성
dataset = ICDAR2015Dataset(root_dir=root_dir, transform=transform)

# DataLoader 생성
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# 데이터 확인
for batch in dataloader:
    images = batch['image']
    boxes = batch['boxes']
    texts = batch['texts']
    print(images.shape)
    print(boxes)
    print(texts)
    break

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/kraggle_Data/ch4_training_images'